In [19]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python
from enum import Enum

from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError, EmailStr
# 导入pydantic对应的模型基类
from pydantic import constr, conint


class GenderEnum(str, Enum):
    """
    性别枚举
    """
    male = "男"
    female = "女"


class User(BaseModel):
    id: int
    name: str = "小卤蛋"
    age: conint(ge=0, le=99)  # 整数范围：0 <= age <= 99
    email: EmailStr
    signup_ts: Optional[datetime] = None
    friends: List[str] = []
    password: constr(min_length=6, max_length=10)  # 字符长度
    phone: constr(pattern=r'^1\d{10}$')  # 正则验证手机号
    sex: GenderEnum  # 枚举验证,只能传: 男和女


if __name__ == '__main__':

    user_data = {
        "id": 123,
        "name": "小卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
        'password': '123456',
        'phone': '13800000000',
        'sex': '男'
    }

    try:
        user = User(**user_data)
        print(f"User id: {user.id}, User name: {user.name}, User email: {user.email}")
        print(user.model_fields_set)
    except ValidationError as e:
        print(f"Validation error: {e.json()}")

User id: 123, User name: 小卤蛋, User email: xiaoludan@example.com
{'phone', 'password', 'age', 'id', 'email', 'signup_ts', 'name', 'sex', 'friends'}


In [2]:
# 不符合模型定义时

if __name__ == '__main__':

    user_data = {
        # "id": 123,
        "name": "小卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
        'password': '123456',
        'phone': '13800000000',
        'sex': '男'
    }

    try:
        user = User(**user_data)
        logger.info(f"User id: {user.id}, User name: {user.name}, User email: {user.email}")
    except ValidationError as e:
        logger.error(f"Validation error: {e.json()}")




2024-07-24 10:17:55.757 | ERROR    | __main__:<module>:21 - Validation error: [{"type":"missing","loc":["id"],"msg":"Field required","input":{"name":"小卤蛋","age":20,"email":"xiaoludan@example.com","signup_ts":"2024-07-19 00:22","friends":["公众号：海哥python","小天才",""],"password":"123456","phone":"13800000000","sex":"男"},"url":"https://errors.pydantic.dev/2.8/v/missing"}]


In [3]:
# 自定义验证
# ! -*-conding: UTF-8 -*-
# @公众号: 海哥python
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError, EmailStr, field_validator
from loguru import logger


def check_name(v: str) -> str:
    """Validator to be used throughout"""
    if not v.startswith("小"):
        raise ValueError("must be startswith 小")
    return v


class User(BaseModel):
    id: int
    name: str = "小卤蛋"
    age: int
    email: EmailStr
    signup_ts: Optional[datetime] = None
    friends: List[str] = []

    validate_fields = field_validator("name")(check_name)

    @field_validator("age")
    @classmethod
    def check_age(cls, age):
        if age < 18:
            raise ValueError("用户年龄必须大于18岁")
        return age

if __name__ == '__main__':
    user_data = {
        "id": 123,
        "name": "小卤蛋",
        "age": 12,
        "email": "xiaoludan@example.com",
        'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
    }
    try:
        user = User(**user_data)
    except ValidationError as e:
        logger.error(f"Validation error: {e.json()}")


2024-07-24 10:18:34.781 | ERROR    | __main__:<module>:46 - Validation error: [{"type":"value_error","loc":["age"],"msg":"Value error, 用户年龄必须大于18岁","input":12,"ctx":{"error":"用户年龄必须大于18岁"},"url":"https://errors.pydantic.dev/2.8/v/value_error"}]


In [4]:
if __name__ == '__main__':
    user_data = {
        "id": 123,
        "name": "大卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
    }
    try:
        user = User(**user_data)
    except ValidationError as e:
        logger.error(f"Validation error: {e.json()}")

2024-07-24 10:18:47.060 | ERROR    | __main__:<module>:13 - Validation error: [{"type":"value_error","loc":["name"],"msg":"Value error, must be startswith 小","input":"大卤蛋","ctx":{"error":"must be startswith 小"},"url":"https://errors.pydantic.dev/2.8/v/value_error"}]


In [31]:
# ! -*-conding: UTF-8 -*-
# @公众号: 海哥python
from datetime import datetime
from typing import List, Optional, Any
from typing_extensions import Self  # 如果python版本不低于3.11，则可以直接从typing中导入Self
from pydantic import BaseModel, ValidationError, EmailStr, field_validator, model_validator
from loguru import logger


def check_name(v: str) -> str:
    """Validator to be used throughout"""
    if not v.startswith("小"):
        raise ValueError("must be startswith 小")
    return v


class User(BaseModel):
    id: int
    name: str = "小卤蛋"
    age: int
    email: EmailStr
    signup_ts: Optional[datetime] = None
    friends: List[str] = []

    validate_fields = field_validator("name")(check_name)

    @field_validator("age")
    @classmethod
    def check_age(cls, age):
        if age < 18:
            raise ValueError("用户年龄必须大于18岁")
        return age

    @model_validator(mode="before")
    @classmethod
    def check_input_data(cls, data: Any) -> Any:
        print("---1----")
        print(data)
        print("----2---")
        return data

    @model_validator(mode="after")
    def check_age_and_name(self) -> Self:
        if self.age < 30 and self.name != "小卤蛋":
            raise ValueError("用户年龄必须小于30岁, 且名字必须为小卤蛋")

        return self


if __name__ == '__main__':
    user_data = {
        "id": 123,
        "name": "小小卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
    }
    try:
        user = User(**user_data)
        # print(user.model_dump())
    except ValidationError as e:
        print(f"Validation error: {e.json()}")

---1----
{'id': 123, 'name': '小小卤蛋', 'age': 20, 'email': 'xiaoludan@example.com', 'signup_ts': '2024-07-19 00:22', 'friends': ['公众号：海哥python', '小天才', b'']}
----2---
Validation error: [{"type":"value_error","loc":[],"msg":"Value error, 用户年龄必须小于30岁, 且名字必须为小卤蛋","input":{"id":123,"name":"小小卤蛋","age":20,"email":"xiaoludan@example.com","signup_ts":"2024-07-19 00:22","friends":["公众号：海哥python","小天才",""]},"ctx":{"error":"用户年龄必须小于30岁, 且名字必须为小卤蛋"},"url":"https://errors.pydantic.dev/2.8/v/value_error"}]


In [27]:
from typing import Annotated

from pydantic import BaseModel, Field, validate_call


class Person(BaseModel):
    name: str = Field(..., min_length=1, max_length=100)
    age: int = Field(..., gt=0, lt=20)


# @validate_call
def greet(person: Person, message: Annotated[str, Field(min_length=1, max_length=100)]):
    print(f"Hello, {person.name}! {message}")


# 正确的调用
greet(Person(name="公众号：海哥python", age=18), "How are you?")
greet(Person(name="公众号：海哥python", age=18), 1)

Hello, 公众号：海哥python! How are you?
Hello, 公众号：海哥python! 1


In [8]:
from typing import Annotated

from pydantic import BaseModel, Field, validate_call


class Person(BaseModel):
    name: str = Field(..., min_length=1, max_length=100)
    age: int = Field(..., gt=0, lt=20)


@validate_call
def greet(person: Person, message: Annotated[str, Field(min_length=1, max_length=100)]):
    print(f"Hello, {person.name}! {message}")


# 错误的调用，将引发验证错误
try:
    greet(Person(name="公众号：海哥python", age=18), 1)
except Exception as e:
    print(e)

1 validation error for greet
1
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type


In [ ]:
# 计算属性


In [39]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError, EmailStr, computed_field
from loguru import logger


class User(BaseModel):
    id: int
    name: str = "小卤蛋"
    age: int
    email: EmailStr
    signup_ts: Optional[datetime] = None
    friends: List[str] = []

    @computed_field  # 计算属性
    @property
    def link(self) -> str:
        return f"尼古拉斯 · {self.name}"


if __name__ == '__main__':

    # logger.info(User.model_json_schema())

    user_data = {
        "id": 123,
        "name": "小卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
    }
    #
    try:
        user = User(**user_data)
        print(user.dict())
        logger.info(f"{user.model_dump()} .... type: {type(user.model_dump())}")
    except ValidationError as e:
        logger.error(f"Validation error: {e.json()}")


2024-07-24 11:42:55.187 | INFO     | __main__:<module>:39 - {'id': 123, 'name': '小卤蛋', 'age': 20, 'email': 'xiaoludan@example.com', 'signup_ts': datetime.datetime(2024, 7, 19, 0, 22), 'friends': ['公众号：海哥python', '小天才', ''], 'link': '尼古拉斯 · 小卤蛋'} .... type: <class 'dict'>


{"id":123,"name":"小卤蛋","age":20,"email":"xiaoludan@example.com","signup_ts":"2024-07-19T00:22:00","friends":["公众号：海哥python","小天才",""],"link":"尼古拉斯 · 小卤蛋"}


In [38]:
# 配置管理

# ! -*-conding: UTF-8 -*-
# @公众号: 海哥python
import os

# 从pydantic模块导入HttpUrl和Field类，用于设置和验证配置数据的类型和约束
from pydantic import HttpUrl, Field
# 从pydantic_settings模块导入BaseSettings类，作为配置类的基类
from pydantic_settings import BaseSettings

# 初始化环境变量，这些环境变量将用于配置应用程序的数据库和API访问
os.environ['DATABASE_HOST'] = "http://baidu.com"
os.environ['DATABASE_USER'] = "公众号：海哥python"
os.environ['DATABASE_PASSWORD'] = "123456abcd"
os.environ['API_KEY'] = "DHKSDsdh*(sdds"


class AppConfig(BaseSettings):
    """
    应用程序配置类，继承自BaseSettings，用于管理应用程序的配置信息。

    Attributes:
        database_host: 数据库主机的URL，必须是一个有效的HTTP或HTTPS URL。
        database_user: 数据库用户的名称，最小长度为5个字符。
        database_password: 数据库用户的密码，最小长度为10个字符。
        api_key: API访问的密钥，最小长度为8个字符。
    """
    # 定义配置项database_host，类型为HttpUrl，确保其为有效的HTTP或HTTPS URL
    database_host: HttpUrl
    # 定义配置项database_user，类型为字符串，默认最小长度为5
    database_user: str = Field(min_length=5)
    # 定义配置项database_password，类型为字符串，默认最小长度为10
    database_password: str = Field(min_length=10)
    # 定义配置项api_key，类型为字符串，默认最小长度为8
    api_key: str = Field(min_length=8)


# 打印配置类的实例化对象的模型信息，用于调试和确认配置的正确性
print(AppConfig().model_dump())


{'database_host': Url('http://baidu.com/'), 'database_user': '公众号：海哥python', 'database_password': '123456abcd', 'api_key': 'DHKSDsdh*(sdds'}


In [11]:
# ! -*-conding: UTF-8 -*-
# @公众号: 海哥python
# 导入Pydantic的HttpUrl和Field类，用于配置验证
from pydantic import HttpUrl, Field
# 导入BaseSettings和SettingsConfigDict类，用于设置配置类的基础行为和配置字典
from pydantic_settings import BaseSettings, SettingsConfigDict


class AppConfig(BaseSettings):
    """
    应用配置类，继承自BaseSettings，用于定义和管理应用的配置项。

    Attributes:
        model_config: 配置模型的设置，用于指定.env文件的位置、编码方式、是否大小写敏感以及额外的配置策略。
        database_host: 数据库主机的URL，必须是一个有效的HTTP或HTTPS URL。
        database_user: 数据库用户的名称，最小长度为5个字符。
        database_password: 数据库用户的密码，最小长度为10个字符。
        api_key: API的密钥，最小长度为8个字符。
    """
    # 定义配置模型的设置，包括.env文件位置、编码、大小写敏感性和额外参数策略
    model_config = SettingsConfigDict(
        env_file=".env",
        env_file_encoding="utf-8",
        case_sensitive=False,
        extra="forbid",
    )

    # 数据库主机的URL，必须是一个有效的HTTP或HTTPS URL
    database_host: HttpUrl
    # 数据库用户的名称，最小长度为5个字符
    database_user: str = Field(min_length=5)
    # 数据库用户的密码，最小长度为10个字符
    database_password: str = Field(min_length=10)
    # API的密钥，最小长度为8个字符
    api_key: str = Field(min_length=8)


# 打印配置类的实例化对象的模型信息，用于调试和确认配置的正确性
print(AppConfig().model_dump())

{'database_host': Url('http://baidu.com/'), 'database_user': '公众号：海哥python', 'database_password': '123456abcd', 'api_key': 'DHKSDsdh*(sdds'}


In [12]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python


from typing import List
from pydantic import BaseModel, conint


class Friend(BaseModel):
    name: str
    age: conint(gt=0, le=99)


class User(BaseModel):
    name: str
    age: conint(gt=0, le=99)
    friends: List[Friend]


# 创建并验证数据
user_data = {
    'name': '公众号：海哥python',
    'age': 30,
    'friends': [{'name': '小卤蛋', 'age': 3}, {'name': '李元芳', 'age': 18}]
}
user = User(**user_data)
print(user)  # name='公众号：海哥python' age=30 friends=[Friend(name='小卤蛋', age=3), Friend(name='李元芳', age=18)]

name='公众号：海哥python' age=30 friends=[Friend(name='小卤蛋', age=3), Friend(name='李元芳', age=18)]


In [13]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python

from pydantic import BaseModel, Field, EmailStr, ValidationError
from typing import List, Optional
from datetime import datetime


class User(BaseModel):
    id: int = Field(..., alias="_id", frozen=True, strict=True)  # 设置别名，创建后id不能被修改，id不能是字符串形式的“123”传入
    name: str = Field(default="小卤蛋", min_length=1, max_length=100)  # 设置默认值，使用 min_length 和 max_length 来限制字符串长度
    age: int = Field(gt=0)  # 支持各类条件验证，这里假设年龄必须大于0
    email: EmailStr
    signup_ts: Optional[datetime] = Field(default_factory=datetime.now, nullable=False, validate_default=True)
    friends: List[str] = Field(default=[], min_items=0)
    passwd: str = Field(min_length=6, max_length=20, exclude=True)  # passwd不会被序列化


if __name__ == '__main__':

    print(User.model_json_schema())

    user_data = {
        "_id": 123,  # 使用别名 _id
        "name": "小卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        # 'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
        "passwd": "123456"
    }

    try:
        user = User(**user_data)
        print(f"创建用户: {user}")
        print(f"转成字典形式： {user.model_dump()} .... type: {type(user.model_dump())}")
        print(f"转成json格式：{user.model_dump_json()} .... type: {type(user.model_dump_json())}")
        print(f"用户属性： User id: {user.id}, User name: {user.name}, User email: {user.email}")
        # user.id = 456   # 这里修改会报错
    except ValidationError as e:
        print(f"Validation error: {e.json()}")

{'properties': {'_id': {'title': ' Id', 'type': 'integer'}, 'name': {'default': '小卤蛋', 'maxLength': 100, 'minLength': 1, 'title': 'Name', 'type': 'string'}, 'age': {'exclusiveMinimum': 0, 'title': 'Age', 'type': 'integer'}, 'email': {'format': 'email', 'title': 'Email', 'type': 'string'}, 'signup_ts': {'anyOf': [{'format': 'date-time', 'type': 'string'}, {'type': 'null'}], 'nullable': False, 'title': 'Signup Ts'}, 'friends': {'default': [], 'items': {'type': 'string'}, 'minItems': 0, 'title': 'Friends', 'type': 'array'}, 'passwd': {'maxLength': 20, 'minLength': 6, 'title': 'Passwd', 'type': 'string'}}, 'required': ['_id', 'age', 'email', 'passwd'], 'title': 'User', 'type': 'object'}
创建用户: id=123 name='小卤蛋' age=20 email='xiaoludan@example.com' signup_ts=datetime.datetime(2024, 7, 24, 10, 23, 36, 402266) friends=['公众号：海哥python', '小天才', ''] passwd='123456'
转成字典形式： {'id': 123, 'name': '小卤蛋', 'age': 20, 'email': 'xiaoludan@example.com', 'signup_ts': datetime.datetime(2024, 7, 24, 10, 23, 36

In [14]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python

from pydantic import BaseModel


class User(BaseModel):
    name: str
    age: int

    class Config:
        str_min_length = 10  # 字符串最小长度
        str_max_length = 20  # 字符串最大长度


user = User(name="John Doe", age=30)

ValidationError: 1 validation error for User
name
  String should have at least 10 characters [type=string_too_short, input_value='John Doe', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/string_too_short

In [15]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError, EmailStr, field_validator, field_serializer
from loguru import logger


class User(BaseModel):
    id: int
    name: str = "小卤蛋"
    age: int
    email: EmailStr
    signup_ts: Optional[datetime] = datetime.now()
    friends: List[str] = []

    @field_validator("age")
    @classmethod
    def check_age(cls, age):
        if age < 18:
            raise ValueError("用户年龄必须大于18岁")
        return age

    @field_serializer('signup_ts')
    def serialize_signup_ts(self, value: datetime) -> str:
        return value.strftime('%Y-%m-%d %H:%M:%S')


if __name__ == '__main__':

    user_data = {
        "id": 123,
        "name": "小卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        # 'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
    }

    try:
        user = User.model_validate(user_data)
        logger.info(f"{user.model_dump()} .... type: {type(user.model_dump())}")
    except ValidationError as e:
        logger.error(f"Validation error: {e.json()}")

2024-07-24 10:24:13.845 | INFO     | __main__:<module>:42 - {'id': 123, 'name': '小卤蛋', 'age': 20, 'email': 'xiaoludan@example.com', 'signup_ts': '2024-07-24 10:24:13', 'friends': ['公众号：海哥python', '小天才', '']} .... type: <class 'dict'>


In [16]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python
from datetime import datetime
from typing import List, Optional, Any
from pydantic import BaseModel, ValidationError, EmailStr, field_validator, field_serializer, model_serializer
from loguru import logger


class User(BaseModel):
    id: int
    name: str = "小卤蛋"
    age: int
    email: EmailStr
    signup_ts: Optional[datetime] = datetime.now()
    friends: List[str] = []

    @field_validator("age")
    @classmethod
    def check_age(cls, age):
        if age < 18:
            raise ValueError("用户年龄必须大于18岁")
        return age

    @field_serializer('signup_ts')
    def serialize_signup_ts(self, value: datetime) -> str:
        return value.strftime('%Y-%m-%d %H:%M:%S')

    @model_serializer
    def serialize_model(self) -> dict[str, Any]:
        return {
            'id': self.id,
            'name': self.name,
            'age': self.age + 1,
            'email': self.email,
            'signup_ts': self.serialize_signup_ts(self.signup_ts),
            'friends': self.friends,
        }


if __name__ == '__main__':

    user_data = {
        "id": 123,
        "name": "小卤蛋",
        "age": 20,
        "email": "xiaoludan@example.com",
        # 'signup_ts': '2024-07-19 00:22',
        'friends': ["公众号：海哥python", '小天才', b''],
    }

    try:
        user = User.model_validate(user_data)
        logger.info(f"{user.model_dump_json()} .... type: {type(user.model_dump_json())}")
    except ValidationError as e:
        logger.error(f"Validation error: {e.json()}")

2024-07-24 10:24:34.546 | INFO     | __main__:<module>:53 - {"id":123,"name":"小卤蛋","age":21,"email":"xiaoludan@example.com","signup_ts":"2024-07-24 10:24:34","friends":["公众号：海哥python","小天才",""]} .... type: <class 'str'>


# 生成文档

In [17]:
#! -*-conding: UTF-8 -*-
# @公众号: 海哥python
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, ValidationError, EmailStr, field_validator
from loguru import logger


class User(BaseModel):
    id: int
    name: str = "小卤蛋"
    age: int
    email: EmailStr
    signup_ts: Optional[datetime] = None
    friends: List[str] = []

    @field_validator("age")
    def check_age(cls, age):
        if age < 18:
            raise ValueError("用户年龄必须大于18岁")
        return age


if __name__ == '__main__':

    logger.info(User.model_json_schema())

2024-07-24 10:25:47.691 | INFO     | __main__:<module>:26 - {'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': '小卤蛋', 'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'email': {'format': 'email', 'title': 'Email', 'type': 'string'}, 'signup_ts': {'anyOf': [{'format': 'date-time', 'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Signup Ts'}, 'friends': {'default': [], 'items': {'type': 'string'}, 'title': 'Friends', 'type': 'array'}}, 'required': ['id', 'age', 'email'], 'title': 'User', 'type': 'object'}
